In [1]:
import pandas as pd
import numpy as np
import pickle as pickle
import os
import time
import mne
import scipy.io as sio

from pathlib import Path
cwd = os.getcwd()
parent = Path(cwd).parent

In [22]:
def raw_to_matlab(sub, verbose = False):        
    raw = mne.io.read_raw_bdf(f"data_original/s{sub}.bdf", preload=True)   
    
    if sub in ['23','24','25','26','27','28','29','30','31','32']: 
        if sub == '23': events = mne.find_events(raw)
        if sub == '29' or sub == '30' or sub == '31' or sub == '32': events = mne.find_events(raw,stim_channel='-0')
        else: events = mne.find_events(raw,stim_channel='')
        raw.pick_channels(['AF3','C3','PO3','Cz','P4'])
        raw.reorder_channels(['AF3','C3','PO3','P4','Cz'])
    else: 
        events = mne.find_events(raw)
        raw.pick_channels(['AF3','T7','Pz','T8','AF4'])
        raw.reorder_channels(['AF3','T7','Pz','AF4','T8'])
    raw, events = raw.copy().resample(128, events=events)
    raw.filter(1., None, fir_design='firwin', skip_by_annotation='edge')
    raw = raw.copy().set_eeg_reference(ref_channels='average')
    
    if verbose: print(raw.info)

    start_baseline = []
    start_trial = []
    start_exp = []
    #23 has normal triggers
    #24 and over have high numbered figures
    if sub not in ['24','25','26','27','28','29','30','31','32']:
        start_exp_trigger = 1
        start_baseline_trigger = 3
        start_trial_trigger = 4
    else:
        start_exp_trigger = 1638145
        start_baseline_trigger = 1638147
        start_trial_trigger = 1638148 
        
    for event in events:
        if sub == '28':
            if event[2] == start_exp_trigger or event[2] == 5832449:
                start_exp.append(event[0])
            if event[2] == start_baseline_trigger or event[2] == 5832451:
                start_baseline.append(event[0])
            if event[2] == start_trial_trigger or event[2] == 5832452:
                start_trial.append(event[0])
        else:
            if event[2] == start_exp_trigger:
                start_exp.append(event[0])
            if event[2] == start_baseline_trigger:
                start_baseline.append(event[0])
            if event[2] == start_trial_trigger:
                start_trial.append(event[0])
    
    data = raw.get_data()
    ratings = pd.read_csv("participant_ratings.csv")
    ratings = ratings[ratings['Participant_id'].isin([int(sub)])]
    channels = np.arange(0,5)

    meta = []
    labels = []
    experiment_num = 1
    for baseline_begin, trial_begin in zip(start_baseline,start_trial):
        chans_meta = []
        for chan in channels:
            baseline = list(data[chan][baseline_begin+(128*2):baseline_begin+(128*5)])
            trial = list(data[chan][trial_begin:trial_begin+(128*60)])
            full_trial = baseline+trial
            chans_meta.append([i*1000000 for i in full_trial])
            #print([i*1000000 for i in full_trial]) #volts to microvolts
            #chans_meta.append(full_trial)            
            #print(np.mean(full_trial))            
        meta.append(chans_meta)             
        valence = ratings.loc[ratings["Experiment_id"]==experiment_num]['Valence'].iloc[0]
        arousal = ratings.loc[ratings["Experiment_id"]==experiment_num]['Arousal'].iloc[0]
        labels.append([valence,arousal])
        experiment_num+=1 
        
    sio.savemat(f"DEAP_5chan_custom_preproc/s{sub}.mat",{"data":meta,"labels":labels})

In [23]:
#subs = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32']
subs = ['28']
#Event IDs: [1638145 1638146 1638147 1638148 1638149 1638151 5832448 5832449 5832451 5832452 5832453]
for sub in subs:
    raw_to_matlab(sub)
    

Extracting EDF parameters from E:\Documents\University\Thesis\Dev\data_original\s28.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1805823  =      0.000 ...  3526.998 secs...
Trigger channel has a non-zero initial value of 1638144 (consider using initial_event=True to detect this event)
297 events found
Event IDs: [1638145 1638146 1638147 1638148 1638149 1638151 5832448 5832449 5832451
 5832452 5832453]
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 423 samples (3.305 sec)

EEG channel type selected for re-referencing
Applying average reference.
Applying a cu